In [1]:
import os
from typing import Dict, List

import psycopg2

DATABASE_URI = os.getenv('DATABASE_URI')
TABLE_PREFIX = "test1_"
ANNOTATION_TABLE_NAME = TABLE_PREFIX + "annotations"
SUBTASK_TABLE_NAME = TABLE_PREFIX + "subtasks"

In [3]:
conn = psycopg2.connect(DATABASE_URI, sslmode="require")
cursor = conn.cursor()

In [4]:
def fetch_annotations(cursor: psycopg2.extensions.cursor) -> Dict[str, Dict]:
    try:
        # Fetch all annotations
        query = f"""   
            SELECT a.video_filename, a.id, a.username, a.created_at, s.start_step, s.end_step, s.subtask, s.time_spent
            FROM {ANNOTATION_TABLE_NAME} a
            JOIN {SUBTASK_TABLE_NAME} s ON a.id = s.annotation_id"""

        cursor.execute(query)
        rows = cursor.fetchall()

        # Group the data by video filename
        annotations = {}
        for row in rows:
            video_filename = row[0]
            annotation_id = row[1]
            username = row[2]
            created_at = row[3]
            start_step = row[4]
            end_step = row[5]
            subtask = row[6]
            time_spent = row[7]

            # create an initial entry for the video filename if it doesn't exist
            # there are multiple annotations for the same video filename
            if video_filename not in annotations:
                annotations[video_filename] = {}

            if annotation_id not in annotations[video_filename]:
                annotations[video_filename][annotation_id] = {
                    "username": username,
                    "created_at": created_at,
                    "subtask_decomposition": [],
                    "time_spent": 0,
                }

            annotations[video_filename][annotation_id]["subtask_decomposition"].append(
                (start_step, end_step, subtask)
            )
            annotations[video_filename][annotation_id]["time_spent"] += time_spent

    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()  # Roll back the transaction in case of error
    finally:
        cursor.close()
        conn.close()

    return annotations


test1_annotations = fetch_annotations(cursor)

In [5]:
test1_annotations

{'Door_20240213-183948_0.mp4': {1: {'username': 'luna',
   'created_at': datetime.datetime(2024, 10, 2, 15, 46, 9, 445322),
   'subtask_decomposition': [(0, 20, 'move to above door handle'),
    (20, 40, 'move down to turn door handle'),
    (41, 56, 'open door'),
    (57, 79, 'return  to home')],
   'time_spent': 657},
  2: {'username': 'kevin_torres',
   'created_at': datetime.datetime(2024, 10, 2, 15, 47, 53, 526493),
   'subtask_decomposition': [(0, 20, 'arm moves over door handle'),
    (21, 39, 'arm pushes down on door handle'),
    (40, 56, 'arm pulls on door handle'),
    (57, 79, 'arm releases door handle and stows away')],
   'time_spent': 413}},
 'Lift_20240220-124804_17.mp4': {3: {'username': 'kevin_torres',
   'created_at': datetime.datetime(2024, 10, 2, 15, 49, 50, 442366),
   'subtask_decomposition': [(0, 12, 'arm moves over block'),
    (12, 15, 'arm grasps block'),
    (16, 28, 'arm lifts the block ')],
   'time_spent': 95},
  4: {'username': 'luna',
   'created_at': d